In [1]:
!nvidia-smi

Thu Feb 19 18:29:31 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch

print("GPU Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))


GPU Available: True
GPU Name: Tesla T4


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

device = 0 if torch.cuda.is_available() else -1
print("Using device:", "GPU" if device == 0 else "CPU")

Using device: GPU


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "facebook/bart-large-cnn"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Please make sure the generation config includes `forced_bos_token_id=0`. 


Loading weights:   0%|          | 0/511 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
def generate_summary(text, max_tokens=120):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=1024
    ).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            num_beams=4,
            no_repeat_ngram_size=3
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return decoded.strip()


In [ ]:
def summary_to_bullets(summary):
    sentences = summary.split(". ")
    bullets = ["• " + s.strip() for s in sentences if s.strip()]
    return "\n".join(bullets)


In [ ]:
sample_text = """
Photosynthesis is a fundamental biological process that sustains life on Earth. It is the mechanism through which green plants, algae, and certain bacteria convert light energy from the sun into chemical energy that can be stored and used to fuel cellular activities. This process not only supports plant life but also forms the foundation of nearly every food chain on the planet.

Photosynthesis takes place inside specialized organelles in plant cells known as chloroplasts. These chloroplasts contain a green pigment called chlorophyll, which absorbs light energy most efficiently in the blue and red wavelengths of the visible spectrum. When sunlight strikes chlorophyll molecules, it excites electrons, initiating a chain of reactions that ultimately result in the production of energy-rich compounds.

The overall process of photosynthesis can be divided into two major stages: the light-dependent reactions and the Calvin cycle. The light-dependent reactions occur in the thylakoid membranes of the chloroplasts. During this stage, light energy is used to split water molecules into hydrogen and oxygen. This reaction releases oxygen as a byproduct, which is then released into the atmosphere. At the same time, energy from sunlight is used to produce ATP and NADPH, which are molecules that store energy temporarily.

The second stage of photosynthesis, known as the Calvin cycle, occurs in the stroma of the chloroplast. Unlike the light-dependent reactions, this stage does not require direct sunlight. Instead, it uses the ATP and NADPH produced earlier to convert carbon dioxide from the air into glucose. Glucose is a simple sugar that plants use as an energy source and as a building block for growth and development.

Glucose produced during photosynthesis serves multiple purposes within the plant. It can be used immediately for energy through cellular respiration, stored as starch for later use, or converted into other organic compounds such as cellulose, which helps build cell walls. Additionally, glucose forms the basis of the food chain because animals and humans obtain energy by consuming plants or other organisms that have eaten plants.

Beyond providing food, photosynthesis plays a critical role in maintaining atmospheric balance. The oxygen released during photosynthesis is essential for aerobic respiration in most living organisms. Without a constant supply of oxygen, complex life forms would not be able to survive. Furthermore, photosynthesis helps regulate carbon dioxide levels in the atmosphere. By absorbing carbon dioxide, plants reduce the concentration of this greenhouse gas and help mitigate the effects of climate change.

Environmental factors significantly influence the rate of photosynthesis. Light intensity, temperature, water availability, and carbon dioxide concentration all affect how efficiently photosynthesis occurs. For example, insufficient light can limit the light-dependent reactions, while extreme temperatures can damage enzymes involved in the Calvin cycle. Similarly, a shortage of water can reduce the plant’s ability to carry out the necessary chemical reactions.

Scientists have studied photosynthesis extensively to improve agricultural productivity and address global food security challenges. By understanding how plants convert light into energy, researchers aim to develop crops that grow faster, produce higher yields, and tolerate environmental stress more effectively. Advances in biotechnology have even explored the possibility of enhancing photosynthetic efficiency to increase food production for a growing global population.

In addition to its biological and ecological importance, photosynthesis has inspired technological innovation. Researchers are developing artificial photosynthesis systems that mimic natural processes to produce clean fuels using sunlight. These systems aim to generate renewable energy sources while reducing reliance on fossil fuels. By studying how nature efficiently captures and converts solar energy, scientists hope to design sustainable energy solutions for the future.

Ultimately, photosynthesis is more than just a plant function; it is a process that sustains ecosystems, supports life, and influences global climate systems. Without photosynthesis, food chains would collapse, oxygen levels would decline, and life as we know it would cease to exist. Its importance cannot be overstated, as it remains one of the most vital processes on Earth.

"""


In [14]:
def adhd_mode(text):

    # Detect long input
    if len(text.split()) > 500:
        summary = summarize_large_text(text)
    else:
        summary = generate_summary(text, max_tokens=120)

    recap = generate_summary(text, max_tokens=50)
    bullets = summary_to_bullets(summary)
    chunks = chunk_text(text)

    return {
        "summary": summary,
        "quick_recap": recap,
        "bullet_points": bullets,
        "focus_chunks": chunks
    }


In [ ]:
def chunk_text(text, sentences_per_chunk=2):
    sentences = text.replace("\n", " ").split(". ")
    chunks = []

    for i in range(0, len(sentences), sentences_per_chunk):
        chunk = ". ".join(sentences[i:i+sentences_per_chunk]).strip()
        if chunk:
            chunks.append(chunk)

    return chunks


In [10]:
chunks = chunk_text(sample_text)

for i, chunk in enumerate(chunks):
    print(f"\n--- Section {i+1} ---")
    print(chunk)



--- Section 1 ---
Photosynthesis is a fundamental biological process that sustains life on Earth. It is the mechanism through which green plants, algae, and certain bacteria convert light energy from the sun into chemical energy that can be stored and used to fuel cellular activities

--- Section 2 ---
This process not only supports plant life but also forms the foundation of nearly every food chain on the planet.  Photosynthesis takes place inside specialized organelles in plant cells known as chloroplasts

--- Section 3 ---
These chloroplasts contain a green pigment called chlorophyll, which absorbs light energy most efficiently in the blue and red wavelengths of the visible spectrum. When sunlight strikes chlorophyll molecules, it excites electrons, initiating a chain of reactions that ultimately result in the production of energy-rich compounds

--- Section 4 ---
The overall process of photosynthesis can be divided into two major stages: the light-dependent reactions and the Calvi

In [ ]:
def split_for_model(text, max_words=300):
    words = text.split()
    chunks = []

    for i in range(0, len(words), max_words):
        chunk = " ".join(words[i:i+max_words])
        chunks.append(chunk)

    return chunks


In [ ]:
def summarize_large_text(text):
    model_chunks = split_for_model(text)
    intermediate_summaries = []

    # Step 1: Summarize each chunk
    for chunk in model_chunks:
        summary = generate_summary(chunk, max_tokens=150)
        intermediate_summaries.append(summary)

    # Step 2: Combine summaries
    combined_summary = " ".join(intermediate_summaries)

    # Step 3: Final compression
    final_summary = generate_summary(combined_summary, max_tokens=150)

    return final_summary


In [16]:
print(adhd_mode(sample_text))

{'summary': 'Photosynthesis is the mechanism through which green plants, algae, and certain bacteria convert light energy from the sun into chemical energy. This process not only supports plant life but also forms the foundation of nearly every food chain on the planet. Without photosynthesis, food chains would collapse, oxygen levels would decline, and life as we know it would cease to exist.', 'quick_recap': 'Photosynthesis is a fundamental biological process that sustains life on Earth. It is the mechanism through which green plants, algae, and certain bacteria convert light energy from the sun into chemical energy. This process not only supports plant life but also forms', 'bullet_points': '• Photosynthesis is the mechanism through which green plants, algae, and certain bacteria convert light energy from the sun into chemical energy\n• This process not only supports plant life but also forms the foundation of nearly every food chain on the planet\n• Without photosynthesis, food cha